# Rating Products and Sorting Reviews in Amazon

Dataframe used in this notebook was extracted from review section of one of Amazon products.
Here are the columns of dataframe and their descriptions.

**reviewerID**: ID of user

**asin**: Unique Product ID

**helpful**: Helpful rating degree

**reviewText**: Review

**overall**: Product Rating given by the reviewer

**summary**: Summary of the review

**unixReviewTime**: Time of review

**reviewTime**: Raw time of review

**day_diff**: Number of days have passed since the day of review

**helpful_yes**: Number of helpful vote for the review

**total_vote**: Total vote for the review

In [1]:
import pandas as pd
import math
import scipy.stats as st
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
pd.set_option("display.width",500)
pd.set_option("display.expand_frame_repr",False)
pd.set_option("display.float_format",lambda x: "%.5f" % x)

In [2]:
df=pd.read_csv(r"C:\Users\ONUR\Downloads\Miuul\Measurement Problems (Week 5)\amazon_review.csv")
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0


In [3]:
df.tail()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
4910,A2LBMKXRM5H2W9,B007WTAJTO,"ZM ""J""","[0, 0]",I bought this Sandisk 16GB Class 10 to use wit...,1.00000,Do not waste your money.,1374537600,2013-07-23,503,0,0
4911,ALGDLRUI1ZPCS,B007WTAJTO,Zo,"[0, 0]",Used this for extending the capabilities of my...,5.00000,Great item!,1377129600,2013-08-22,473,0,0
4912,A2MR1NI0ENW2AD,B007WTAJTO,Z S Liske,"[0, 0]",Great card that is very fast and reliable. It ...,5.00000,Fast and reliable memory card,1396224000,2014-03-31,252,0,0
4913,A37E6P3DSO9QJD,B007WTAJTO,Z Taylor,"[0, 0]",Good amount of space for the stuff I want to d...,5.00000,Great little card,1379289600,2013-09-16,448,0,0
4914,A8KGFTFQ86IBR,B007WTAJTO,Zza,"[0, 0]",I've heard bad things about this 64gb Micro SD...,5.00000,So far so good.,1388620800,2014-02-01,310,0,0


In [4]:
df.shape
# 4915 comments have been made for this particular product.
# 12 variables

(4915, 12)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      4915 non-null   object 
 1   asin            4915 non-null   object 
 2   reviewerName    4914 non-null   object 
 3   helpful         4915 non-null   object 
 4   reviewText      4914 non-null   object 
 5   overall         4915 non-null   float64
 6   summary         4915 non-null   object 
 7   unixReviewTime  4915 non-null   int64  
 8   reviewTime      4915 non-null   object 
 9   day_diff        4915 non-null   int64  
 10  helpful_yes     4915 non-null   int64  
 11  total_vote      4915 non-null   int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 460.9+ KB


In [6]:
df.isnull().sum()

reviewerID        0
asin              0
reviewerName      1
helpful           0
reviewText        1
overall           0
summary           0
unixReviewTime    0
reviewTime        0
day_diff          0
helpful_yes       0
total_vote        0
dtype: int64

In [7]:
df.nunique()

reviewerID        4915
asin                 1
reviewerName      4594
helpful             42
reviewText        4912
overall              5
summary           3885
unixReviewTime     690
reviewTime         690
day_diff           690
helpful_yes         23
total_vote          26
dtype: int64

In [8]:
df.describe([0.1,0.2,0.4,0.6,0.8,0.9,0.99])

,overall,unixReviewTime,day_diff,helpful_yes,total_vote
count,4915.00000,4915.00000,4915.00000,4915.00000,4915.00000
mean,4.58759,1379465001.66836,437.36704,1.31109,1.52146
std,0.99685,15818574.32275,209.43987,41.61916,44.12309
min,1.00000,1339200000.00000,1.00000,0.00000,0.00000
10%,4.00000,1356825600.00000,167.00000,0.00000,0.00000
20%,4.00000,1362528000.00000,248.00000,0.00000,0.00000
40%,5.00000,1375142400.00000,361.00000,0.00000,0.00000
50%,5.00000,1381276800.00000,431.00000,0.00000,0.00000
60%,5.00000,1386806400.00000,497.40000,0.00000,0.00000
80%,5.00000,1394582400.00000,638.00000,0.00000,0.00000


# Rating Product

In [9]:
# Firstly, let's compute raw average rating by using average function on overall
df.overall.mean()

4.587589013224822

In [10]:
# Now proceeding to Time Based Weighted Average
# To apply this method we first need a review Time column to have datetime datatype.
df["reviewTime"]=pd.to_datetime(df["reviewTime"])
df.dtypes

reviewerID                object
asin                      object
reviewerName              object
helpful                   object
reviewText                object
overall                  float64
summary                   object
unixReviewTime             int64
reviewTime        datetime64[ns]
day_diff                   int64
helpful_yes                int64
total_vote                 int64
dtype: object

In [11]:
# A date which represents analysis date must be determined. To do that we take the latest date in reviewTime column
today_time=df["reviewTime"].max()
today_time

Timestamp('2014-12-07 00:00:00')

In [12]:
# Recency
df["recency"]=(today_time-df["reviewTime"]).dt.days
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,recency
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0,137
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,408
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0,714
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,381
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0,512


In [13]:
df.recency.quantile([0.1,0.25,0.4,0.5,0.6,0.8,0.9,1])

0.10000    166.00000
0.25000    280.00000
0.40000    360.00000
0.50000    430.00000
0.60000    496.40000
0.80000    637.00000
0.90000    707.00000
1.00000   1063.00000
Name: recency, dtype: float64

In [14]:
# Defining a function that can compute overall average rating for given time intervals and their individual weights.
def time_based_weighted_average(dataframe,p1=270,p2=420,p3=600,p4=1063,w1=30,w2=27,w3=24,w4=19):
    """
    p1,p2,p3,p4: Represents the period limits according to which reviews will be grouped
    w1,w2,w3,w4: Represents the weights allocated to each time interval
    dataframe: Dataframe to be provided for the function
    """
    return dataframe.loc[dataframe["recency"]<p1,"overall"].mean() * w1/100 +\
        dataframe.loc[(dataframe["recency"]>p1)&(dataframe["recency"]<=p2),"overall"].mean() * w2/100 +\
        dataframe.loc[(dataframe["recency"]>p2)&(dataframe["recency"]<=p3),"overall"].mean() * w3/100 +\
        dataframe.loc[(dataframe["recency"]>p3)&(dataframe["recency"]<=p4),"overall"].mean() * w4/100

In [15]:
print("Time Based Weighted Rating Average: \n",time_based_weighted_average(df))
print("Raw Rating Average \n", df["overall"].mean())
# There is a slight difference between two average stemming 
# from the temporal features of Time Based Weighted Average.

Time Based Weighted Rating Average: 
 4.6052898884829805
Raw Rating Average 
 4.587589013224822


In [16]:
# Let's examine the Rating Average for each period of time

# Average overall for last 270 days
df.loc[df["recency"]<270,"overall"].mean()

4.686592655849701

In [17]:
# Average overall between last 270 and 420 days
df.loc[(df["recency"]>270)&(df["recency"]<=420),"overall"].mean()

4.676592224979322

In [18]:
# Average overall between last 420 and 600 days
df.loc[(df["recency"]>420)&(df["recency"]<=600),"overall"].mean()

4.549349655700077

In [19]:
# Average overall between last 600 and 1063 days
df.loc[(df["recency"]>600)&(df["recency"]<=1063),"overall"].mean()

4.4462540716612375

* **As can be seen above there is an upwards trends for overall rating as the reviews gets more recent**

# Sorting Reviews

In [20]:
df.sample(4)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,recency
902,A3EF29X3EZJ8YV,B007WTAJTO,Charles Soccerleg,"[0, 0]",I put it in my tablet and so far nothing to co...,4.00000,nothing to complain about,1370131200,2013-02-06,670,0,0,669
2962,A10W9RI5KC0KR,B007WTAJTO,MarcP,"[0, 0]",This device was bought for a Galaxy Tab 10....,5.00000,Great,1360195200,2013-07-02,524,0,0,523
2612,A1Q50RSQ0Q9CTN,B007WTAJTO,kc2pxp,"[0, 0]",After reading many reviews I decided to purcha...,5.00000,Purchased for galaxy S3 phone,1352851200,2012-11-14,754,0,0,753
4214,A1Y4UXTB7Y17OG,B007WTAJTO,skn777,"[0, 0]",Arrived earlier than expected. Been using it a...,5.00000,More storage for Android phone!,1379635200,2013-09-20,444,0,0,443


In [21]:
# There are up and down options for each review. If a user find a comment helpful, they vote "up".
# On the contrary, a comment were to be found not helpful, user vote "down".
# Dataframe already has the number of "up"s for each review in helpful_yes column. 
# Number of "down"s can be found by extracting helpful_yes column from total_vote column.
df["helpful_no"]=df["total_vote"]-df["helpful_yes"]
df.sample(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,recency,helpful_no
2569,A25NNMF17T7HVG,B007WTAJTO,justing6,"[0, 0]",With this card I picked up for $35 amazon's bl...,4.00000,Best accessory for my galaxy note II,1358294400,2013-01-16,691,0,0,690,0
3541,A2THH8QNX2IOPW,B007WTAJTO,Patrick Cantrell,"[0, 0]",Extremely easy install for Samsung Galaxy Tab ...,5.00000,Works great for my Galaxy Tab 3.,1388966400,2014-06-01,190,0,0,189,0
2542,A26UOD4OQCLCRH,B007WTAJTO,"Juan C. Yunis ""Juan C. Yunis""","[0, 0]",This was a good investment for my galaxy note ...,5.00000,Fast and Reliable,1358208000,2013-01-15,692,0,0,691,0
1030,A3UXPRFNG8GMR8,B007WTAJTO,"ConstantLearning ""noahsmom""","[0, 0]",Works great in the Samsung Galaxy S3 phone. I ...,5.00000,Works perfectly for us in the Samsung Galaxy S3.,1383609600,2013-05-11,576,0,0,575,0
2912,A20GGRHRQ56XLR,B007WTAJTO,Luv to Shop,"[0, 0]",Definitely needed more memory for my phone. T...,5.00000,Yay,1395014400,2014-03-17,266,0,0,265,0


In [22]:
# First method used for sorting products is up and down difference score
def score_pos_neg_diff(up,down):
    return up-down
df["score_pos_neg_diff"]=score_pos_neg_diff(df["helpful_yes"],df["helpful_no"])
df.sample(2)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,recency,helpful_no,score_pos_neg_diff
2063,A2BLDYW9BXLRI,B007WTAJTO,Irritated,"[0, 0]",Phone is a galaxy s3 from virgin. Still gettin...,4.00000,Good fast card.,1380672000,2013-02-10,666,0,0,665,0,0
556,A1UBBRNBLEU26G,B007WTAJTO,"Beau Hunter ""Beau""","[0, 0]",Figured there was not going to be any issues w...,5.00000,Used in Samsung Galaxy,1389312000,2014-10-01,68,0,0,67,0,0


In [23]:
# Second method to use for sorting products is average rating of helpful_yes
def score_average_rating(up,down):
    if (up+down) ==0:
        return 0
    return up/(up+down)
df["score_average_rating"]=df.apply(lambda df: 
                                    score_average_rating(df["helpful_yes"], df["helpful_no"]),axis=1)
df.sample(2)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,recency,helpful_no,score_pos_neg_diff,score_average_rating
2387,A2UDFXKJKPL8G5,B007WTAJTO,Joey J.,"[0, 0]",Ordered this to go with the tablet I ordered f...,5.00000,Great deal!,1379980800,2013-09-24,440,0,0,439,0,0,0.00000
4577,ATIMW8SYGAASW,B007WTAJTO,Todd F. Fogleman,"[0, 0]","fast transfers and retrieval of data on phone,...",5.00000,fast,1358985600,2013-01-24,683,0,0,682,0,0,0.00000


In [24]:
# Third and most efficient method of them all is Wilson Lower Bound Method
def wilson_lower_bound(up, down, confidence=0.95):
    """
    Parameters
    ----------
    up: int
        up count
    down: int
        down count
    confidence: float
        confidence

    Returns
    -------
    wilson score: float
    """
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)
df["wilson_lower_bound"]=df.apply(lambda x: wilson_lower_bound(x["helpful_yes"],x["helpful_no"]),axis=1)

In [25]:
df.sample(3)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,recency,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
3571,A2K7DSA1ZS6GLM,B007WTAJTO,PCEddie,"[0, 0]",I purchased the SanDisk Ultra 64GB MicoSDXC me...,5.00000,It Works,1376179200,2013-11-08,395,0,0,394,0,0,0.00000,0.00000
3815,A1VD8OR0XWKV8S,B007WTAJTO,Ricardo Ruggiero,"[0, 0]","Easy to install, good performance, can be used...",5.00000,Good,1397952000,2014-04-20,232,0,0,231,0,0,0.00000,0.00000
4855,A3DVTGEG3A6RDT,B007WTAJTO,ws,"[0, 0]",Slower than expected transfer rates when used ...,3.00000,Lower than expected transfer rates,1401148800,2014-05-27,195,0,0,194,0,0,0.00000,0.00000


In [26]:
# Picking only the columns we need to show to clean up the table
df=df[["overall","reviewTime","helpful_yes","helpful_no","total_vote","score_pos_neg_diff",
       "score_average_rating","wilson_lower_bound"]]
df.sample(4)

,overall,reviewTime,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
4263,5.00000,2014-03-04,0,1,1,-1,0.00000,0.00000
3854,5.00000,2012-11-30,0,0,0,0,0.00000,0.00000
3784,5.00000,2014-01-05,0,0,0,0,0.00000,0.00000
4194,5.00000,2013-03-04,0,0,0,0,0.00000,0.00000


In [27]:
# Sorting commennts by wilson_lower_bound to see which ones are the most helpful.
df.sort_values(by="wilson_lower_bound",ascending=False).head(10)

,overall,reviewTime,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,5.00000,2013-01-05,1952,68,2020,1884,0.96634,0.95754
3449,5.00000,2012-09-26,1428,77,1505,1351,0.94884,0.93652
4212,1.00000,2013-05-08,1568,126,1694,1442,0.92562,0.91214
317,1.00000,2012-02-09,422,73,495,349,0.85253,0.81858
4672,5.00000,2014-07-03,45,4,49,41,0.91837,0.80811
1835,5.00000,2014-02-28,60,8,68,52,0.88235,0.78465
3981,5.00000,2012-10-22,112,27,139,85,0.80576,0.73214
3807,3.00000,2013-02-27,22,3,25,19,0.88000,0.70044
4306,5.00000,2012-09-06,51,14,65,37,0.78462,0.67033
4596,1.00000,2012-09-22,82,27,109,55,0.75229,0.66359


***Wilson Lower Bound score indicates the degree of helpfulness to the potential***
***customer in the process of deciding to buy the product***

In [28]:
## FİRST CONCLUSION
# Reviews like index 317 OR 4212 ,third and fourth rows on this query, has a lower up/total_vote rate compared to other but due to amount of voting
# this review becomes one of the most helpful reviews of all. We can make an inference that when it comes to helpfulness
# postiveness (positiveness=up/(up+down)) doesn't matter much. As long as the review helps potential customers, there is no difference
# in review being positive or negative overall. By the way, usually we review the interval from 1-3 stars to be negative while 4-5 stars
# are considered to be positive.

## SECOND CONCLUSION
# Social proof, people will pay heed to masses opinion, proves to be true again in this query where reviews with 
# big amount of votes placed high in the query. As can be seen below score_average_rating is highly correlated with
# the helpfulness of the review.

In [29]:
df.corr()

,overall,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
overall,1.00000,-0.03121,-0.06614,-0.03547,-0.02624,-0.15300,-0.13855
helpful_yes,-0.03121,1.00000,0.59275,0.99730,0.99661,0.10410,0.33813
helpful_no,-0.06614,0.59275,1.00000,0.65029,0.52444,0.08458,0.27240
total_vote,-0.03547,0.99730,0.65029,1.00000,0.98787,0.10590,0.34378
score_pos_neg_diff,-0.02624,0.99661,0.52444,0.98787,1.00000,0.10141,0.32963
score_average_rating,-0.15300,0.10410,0.08458,0.10590,0.10141,1.00000,0.87318
wilson_lower_bound,-0.13855,0.33813,0.27240,0.34378,0.32963,0.87318,1.00000
